In [ ]:
import os
COLAB = True
work_dir = "/home/changl25/ziplora-pytorch"
os.chdir(work_dir)
!pwd
import sys
from pathlib import Path

# Playground

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np 
import torch
from diffusers import AutoencoderKL
from transformers import AutoTokenizer
MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"
device = "cuda" if torch.cuda.is_available() else "cpu"
weight_dtype = torch.float16
rank=64
%load_ext autoreload
%autoreload 2

from inverse_ziplora_comb_separate import utils
from inverse_ziplora_comb_separate.pipeline_stable_diffusion_xl_seperate import StableDiffusionXLSeperatePipeline


In [ ]:
from train_inverse_ziplora_layer_column import import_model_class_from_model_name_or_path

tokenizer_one = AutoTokenizer.from_pretrained(
    MODEL_ID,
    subfolder="tokenizer",
    use_fast=False,
)
tokenizer_two = AutoTokenizer.from_pretrained(
    MODEL_ID,
    subfolder="tokenizer_2",
    use_fast=False,
)
text_encoder_cls_one = import_model_class_from_model_name_or_path(
    MODEL_ID, revision=None
)
text_encoder_cls_two = import_model_class_from_model_name_or_path(
    MODEL_ID, subfolder="text_encoder_2", revision=None
)
text_encoder_one = text_encoder_cls_one.from_pretrained(
    MODEL_ID, subfolder="text_encoder",
)
text_encoder_two = text_encoder_cls_two.from_pretrained(
    MODEL_ID, subfolder="text_encoder_2",
)
text_encoder_one.requires_grad_(False)
text_encoder_one.to(device, dtype=torch.float16)

text_encoder_two.requires_grad_(False)
text_encoder_two.to(device, dtype=torch.float16)
tokenizers = [tokenizer_one, tokenizer_two]
text_encoders = [text_encoder_one, text_encoder_two]

vae = AutoencoderKL.from_pretrained(
            MODEL_ID,
            subfolder="vae",
            revision=None
    )

In [ ]:
def log_validation(pipeline, prompt, prompt_content, prompt_style, seed=0):
    generator = torch.Generator(device=device).manual_seed(seed)
    # Currently the context determination is a bit hand-wavy. We can improve it in the future if there's a better
    # way to condition it. Reference: https://github.com/huggingface/diffusers/pull/7126#issuecomment-1968523051
    if pipeline.__class__.__name__ == 'StableDiffusionXLSeperatePipeline':
        pipeline_args = {"prompt": prompt, 
                        "prompt_content": prompt_content, 
                        "prompt_style": prompt_style}
    else: 
        pipeline_args = {"prompt": prompt}
    print(pipeline_args)    
    images = [pipeline(**pipeline_args, generator=generator, num_inference_steps=50).images[0] for _ in range(4)]
    return images

In [ ]:
def show_img(images):
    np_images = np.hstack([np.asarray(img) for img in images])
    plt.figure(figsize=(10,10))
    plt.imshow(np_images)
    plt.axis("off")
    plt.show()

## In-image Generation

## Cross-image Generation